In [9]:

import numpy as np

# filestn = '/datastore/GLOBALWATER/CommonData/EMDNA_new/stndata_whole.npz'
# filereg = '/datastore/GLOBALWATER/CommonData/EMDNA_new/stn_regression/daily_regression_stn.npz'
# file_nearstn = '/datastore/GLOBALWATER/CommonData/EMDNA_new/stn_regression/nearstn_catalog.npz'

filestn = '/Users/localuser/Research/EMDNA/stndata_whole.npz'
filereg = '/Users/localuser/Research/EMDNA/regression/daily_regression_stn.npz'
file_nearstn = '/Users/localuser/Research/EMDNA/regression/weight_nearstn.npz'
nearnum = 10  # compare target station with its nearby stations to determine climatological anomaly

########################################################################################################################

# precipitation
var = 'prcp'
d = np.load(filestn)
dstn = d[var + '_stn']
stn_lle = d['stn_lle']
d = np.load(filereg)
dreg = d[var]

d = np.load(file_nearstn)
if var == 'prcp':
    nearloc = d['near_stn_prcpLoc']
else:
    nearloc = d['near_stn_tempLoc']

# calculate two types of ratio
nstn, ntimes = np.shape(dstn)
dstnm = np.nanmean(dstn, axis=1)
dregm = np.nanmean(dreg, axis=1)
ratio_vsreg = np.abs(dstnm) / np.abs(dregm)
ratio_vsnear = np.nan * np.zeros(nstn)
for i in range(nstn):
    if np.isnan(dstnm[i]):
        continue
    zz = nearloc[i, 0:nearnum]
    zz = zz[zz > -1]
    ratio_vsnear[i] = abs(dstnm[i] / np.nanmean(dstnm[zz]))

# determine toxic stations: two-tailed percentile check
# the thresholds are determined by trial-and-error
# for precipitation, stations with too dry precipitation is more common. besides, we prefer to keep more stations
# with heavy precipitation to benefit extreme precipitation estimation
p1 = np.nanpercentile(ratio_vsreg, 99.9)
p2 = np.nanpercentile(ratio_vsnear, 99.9)
p3 = np.nanpercentile(ratio_vsreg, 1)
p4 = np.nanpercentile(ratio_vsnear, 1)

# toxic_prcp=np.where(((ratio_vsreg<p3) & (ratio_vsnear<p4)))[0]
toxic_prcp = np.where(((ratio_vsreg > p1) & (ratio_vsnear > p2)) |
                      ((ratio_vsreg < p3) & (ratio_vsnear < p4)))[0]

del dstn, dregm

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: Mean of empty slice


IndexError: too many indices for array

In [37]:
vart = ['tmean', 'trange']
toxic_temp = [' '] * 2
for v in range(len(vart)):
    var = vart[v]
    d = np.load(filestn)
    dstn = d[var + '_stn']
    stn_lle = d['stn_lle']
    d = np.load(filereg)
    dreg = d[var]

    d = np.load(file_nearstn)
    if var == 'prcp':
        nearloc = d['near_stn_prcpLoc']
    else:
        nearloc = d['near_stn_tempLoc']

    nstn, ntimes = np.shape(dstn)
    dstnm = np.nanmean(dstn, axis=1)
    dregm = np.nanmean(dreg, axis=1)
    diff_vsreg = np.abs(dstnm - dregm)
    diff_vsnear = np.nan * np.zeros(nstn)
    for i in range(nstn):
        if np.isnan(dstnm[i]):
            continue
        zz = nearloc[i, 0:nearnum]
        zz = zz[zz > -1]
        diff_vsnear[i] = np.abs(dstnm[i] - np.mean(dstnm[zz]))

    p1 = np.nanpercentile(diff_vsreg, 99)
    p2 = np.nanpercentile(diff_vsnear, 95)
    toxic_temp[v] = np.where((diff_vsreg > p1) & (diff_vsnear > p2))[0]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in greater


In [38]:
print(len(toxic_temp[0]),len(toxic_temp[1]))

64 91


In [39]:
toxic_temp=np.union1d(toxic_temp[0],toxic_temp[1])
288/np.sum(~np.isnan(dstnm))

0.014654996946875637

In [31]:
np.savez_compressed('toxic_temp.npz',toxic_temp=toxic_temp)

In [36]:
ind_toxic1=toxic_temp[1]
for i in range(len(ind_toxic1)):
    itar=ind_toxic1[i]
    inear=nearloc[itar]
    inear=inear[inear>-1]
    clim1=min(dstnm[inear])-5
    clim2=max(dstnm[inear])+5
    plt.scatter(stn_lle[inear,1],stn_lle[inear,0],50,dstnm[inear],cmap='jet')
    plt.clim([clim1,clim2])
    plt.scatter(stn_lle[itar,1],stn_lle[itar,0],200,dstnm[itar],cmap='jet',edgecolors=[0,0,0])
    plt.clim([clim1,clim2])
    plt.colorbar()
    plt.show()

TypeError: object of type 'numpy.int64' has no len()